In [59]:
import pandas as pd
import numpy as np 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from xgboost import XGBClassifier
import optuna
from optuna.samplers import TPESampler

In [60]:
dataset = pd.read_csv('../db/AmesHousing.csv', sep='\t')
localCopy=dataset.copy(deep=True)

In [61]:
#SEPARATION OF CATEGORICAL AND NUMERICAL FEATURES

#AmesExplanations.txt indicates a separation of the data in 4 types:
# Continuous - numerical
# Nominal - categorical
# Ordinal - usually categorical, numerical if order matters
# Discrete - usually numerical, categorical if it's an ID

cat_features = ['MS SubClass','MS Zoning','Street','Alley',
                'Lot Shape','Land Contour','Utilities','Lot Config',
                'Land Slope','Neighborhood','Condition 1','Condition 2',
                'Bldg Type','House Style','Roof Style','Roof Matl',
                'Exterior 1st','Exterior 2nd','Mas Vnr Type','Foundation',
                'Heating','Central Air','Electrical','Garage Type',
                'Paved Drive','Misc Feature','Sale Condition']

num_features = ['Lot Frontage','Lot Area','Overall Qual','Overall Cond',
                'Year Built','Year Remod/Add','Mas Vnr Area','Exter Qual',
                'Exter Cond','Bsmt Qual','Bsmt Cond','Bsmt Exposure',
                'BsmtFin Type 1', 'BsmtFin SF 1','BsmtFin Type 2','BsmtFin SF 2',
                'Bsmt Unf SF','Total Bsmt SF','Heating QC','1st Flr SF','2nd Flr SF',
                'Low Qual Fin SF','Gr Liv Area','Bsmt Full Bath','Bsmt Half Bath',
                'Full Bath','Half Bath','Bedroom','Kitchen','Kitchen Qual','TotRmsAbvGrd',
                'Functional','Fireplaces','Fireplace Qu',
                'Garage Yr Blt','Garage Finish','Garage Cars','Garage Area','Garage Qual',
                'Garage Cond','Wood Deck SF','Open Porch SF','Enclosed Porch','3-Ssn Porch',
                'Screen Porch','Pool Area','Pool QC','Fence','Misc Val','Mo Sold','Yr Sold',
                'Sale Type']


# Features that describe a grade (ex. Quality: 1-10 or Poor/Average/Excellent) will be treated as
# numeric values. There are string values as well; these will be mapped to numbers so that the model
# understands them

# to be mapped to numbers: Exter Qual, Exter Cond, Bsmt Qual, Bsmt Cond, Bsmt Exposure,
# BsmtFin Type 1, BsmtFinType 2, HeatingQC, KitchenQual, Functional, FireplaceQu, 
# Garage Finish, Garage Qual, Garage Cond, Pool QC, Fence, Sale Type

ordinal_maps = {'Exter Qual': {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Exter Cond': {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Bsmt Qual': {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Bsmt Cond': {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Bsmt Exposure': {'NA':0, 'No':1, 'Mn':2, 'Av':3, 'Gd':4},
                'BsmtFin Type 1': {'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
                'BsmtFin Type 2': {'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
                'Heating QC': {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Kitchen Qual': {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Functional': {'Sal':1, 'Sev':2, 'Maj2':3, 'Maj1':4, 'Mod':5, 'Min2':6, 'Min1':7, 'Typ':8},
                'Fireplace Qu': {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Garage Finish': {'NA':0, 'Unf':1, 'RFn':2, 'Fin':3},
                'Garage Qual': {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Garage Cond': {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Pool QC': {'NA':0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4},
                'Fence': {'NA':0, 'MnWw':1, 'GdWo':2, 'MnPrv':3, 'GdPrv':4},
                'Sale Type': {'Oth':1, 'ConLD':2, 'ConLI':3, 'ConLw':4, 'Con':5, 'COD':6, 'New':7, 'VWD':8, 'CWD':9, 'WD':10}
                }

# apply the mappings to the strings
for col,mapping in ordinal_maps.items():
    if col in localCopy.columns:
        localCopy[col] = localCopy[col].map(mapping)

In [62]:
#FEATURE ENGINEERING: Deletion

#dropping the PID , Order Column
# Yr Sold, Sale Price, Year Built, Total Bsmt SF, Gr Liv Area because they have a high correlation with 
# other features, we are gonna keep them and delete the others
localCopy.drop(columns=['PID','Order','Overall Qual', 'Garage Yr Blt', '1st Flr SF','TotRms AbvGrd', 'Garage Cars'],inplace=True)
localCopy.drop(index=[1341,1497,1577,2236],inplace=True)

#dropping columns with many null values
for column in localCopy:
    if localCopy[column].isna().sum()>2000:
        localCopy.drop(column, axis=1, inplace=True)

#After all deletions, remove the deleted features in the 2 cat/num separation arrays as well
cat_features = [col for col in cat_features if col in localCopy.columns]
num_features = [col for col in num_features if col in localCopy.columns]
#Also do the same for the ordinal_maps
ordinal_maps = {col: mapping for col,mapping in ordinal_maps.items() if col in localCopy.columns}

In [63]:
#Separate the target variable from the rest
indexOfTV = localCopy.columns.get_loc("SalePrice")
X = localCopy.iloc[:,:indexOfTV]  #all rows and all columns except for the TV column
Y = localCopy.iloc[:,indexOfTV]  #column of the Target Variable with all the rows (":")

# compute the quartiles 
# step 1: copy/sort the data
Y_copy = Y.copy(deep=True).sort_values(ascending=True)
length = len(Y_copy)
# a ranges vector that defines the classes: e.g. for the class 1 we must look up to be between the 
# ranges[0] and ranges[1] => for each class 'i' the range is defined by (ranges[i-1], ranges[i])
min = np.min(np.array(Y)); max = np.max(np.array(Y))
ranges = [min, -1, -1, -1, max]

# length is even then:
# Q2 = mean(length/2, length/2-1)
Q2 = np.mean(np.array(Y_copy.iloc[(length//2)-1:length//2+1]))

# split the Series and compute the Q1, Q3
cond = Y_copy <= Q2
half1, half2 = Y_copy[cond], Y_copy[~cond]

Q1 = np.median(np.array(half1))
Q3 = np.median(np.array(half2))

ranges[1] = Q1
ranges[2] = Q2 
ranges[3] = Q3

# change the target varaible to a categorical format
classes = ["cheap", "affordable", "expensive", "billionaire type of shit"]
cat_y: list[str] = []
for y in Y:
    if y <= ranges[1]:
        cat_y.append(classes[0])
    elif y <= ranges[2]:
        cat_y.append(classes[1])
    elif y <= ranges[3]:
        cat_y.append(classes[2])
    else:
        cat_y.append(classes[3])

Y = pd.Series(cat_y)

In [64]:
#FEATURE ENGINEERING: Scaling

#Scalam doar features numerice
scaler = MinMaxScaler()
scaled_data = X.copy(deep=True)
scaled_data[num_features] = scaler.fit_transform(X[num_features])

X = scaled_data

In [65]:
# One-Hot Encoding for the categorical variables
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

encodedData = encoder.fit_transform(X[cat_features])
encodedDataFrame = pd.DataFrame(encodedData, columns=encoder.get_feature_names_out(cat_features))

X = pd.concat([X, encodedDataFrame], axis=1)
X = X.drop(cat_features, axis=1)
X.drop(index=[1341,1497,1577,2236], axis=0, inplace=True)


# Ordinal Encoding for the hand-made classes of the target variable
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[classes])

Y = encoder.fit_transform(pd.DataFrame(Y))
Y = pd.DataFrame(Y)

In [66]:
# XGBoost

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

model = XGBClassifier(
    num_class=len(set(Y)), 
    eval_metric='mlogloss',
    max_depth=4,
    learning_rate=0.1,
    n_estimators=90,
    random_state=42
)

model.fit(X_train, Y_train)

# predicting 
y_pred = model.predict(X_test)
print(f" Accuracy of prediction: {accuracy_score(Y_test, y_pred)}")

 Accuracy of prediction: 0.8088737201365188


In [67]:
#--------------------------------------------------------------------------------------------
# LAB 3

# Optuna Bayesian Optimisation: incercam intr-un numar dat de trialuri valori diferite
# pt niste hiperparametrii alesi.
# Pt Optuna, ne facem o functie (objective) in care se antreneaza modelul xgb. Functia e apelata 
# in fiecare trial si returneaza o valoare (aici: accuracy).
# Optuna schimba valorile hiperparametrilori alesi ca sa ajunga la un return al functiei mai preferabil


def objective(trial):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
    
    dtrain = xgb.DMatrix(X_train, label=Y_train)
    dvalid = xgb.DMatrix(X_test, label=Y_test)

    param = {
    "objective": "multi:softprob",  # returns probabilities for each class (ex. [0.2, 0.5, 0.1, 0.2]; explicatie mai buna jos unde i y_pred_tuned)
    "eval_metric": "mlogloss",      # multi-class log loss
    "num_class": 4,                 # total number of classes
    "verbosity": 0,
    "subsample": trial.suggest_float("subsample", 0.4, 1.0),   # nr randuri alese aleatoriu pt fiecare arbore
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),   # nr features alese aleatoriu pt fiecare arbore
    "min_child_weight": trial.suggest_float("min_child_weight", 1, 10),      # controleaza minimul marimii unei frunze
    }
    # unde apare trial.suggest_float: # ii spunem la optuna sa incerce pt hiperparametrii 
    # respectivi valori intre cele 2 numere (ex. 0.4 si 1.0)
    

    gbm = xgb.train(param, dtrain,
        num_boost_round=200,
        evals=[(dvalid, "validation")],
        early_stopping_rounds=20,
        verbose_eval=False,)

    preds = gbm.predict(dvalid)
    pred_labels = np.argmax(preds, axis=1)
    accuracy = accuracy_score(Y_test, pred_labels)
    return accuracy

In [68]:
# Pt optuna facem un obiect "study"; asta se va ocupa de hyperparameter tuning.
# In obiect se vor salva toate trial-urile, parametrii folositi per trial si scorurile 
# obtinute per trial.

study = optuna.create_study(
    direction="maximize",      # optuna cauta cea mai *mare* valoare pt ce vrem. ("ce vrem" - aici: accuracy (returnat de functia objective); in alta situatie, direction="minimize" se putea folosi de ex. pt loss function)
    sampler=TPESampler(),      # metoda eficienta care spune pe ce valori sa lucreze in urmatorul trial
    pruner=optuna.pruners.MedianPruner(),   # metoda de early stopping pt a economisi timp
    study_name="Optune study object",       # numele care il dam obiectului
    load_if_exists=False       # la fiecare rulare creeaza un obiect nou; IMPORTANT pt ca daca rulezi mereu cu acelasi obiect inca un set de trialuri, tu mai ai si trialurile de inainte salvate care, daca ai schimbat ceva mai sus, sunt gresite
)

[I 2025-11-07 18:18:39,292] A new study created in memory with name: Optune study object


A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object


In [69]:
# study.enqueue_trial: Ii spunem la optuna "Neaparat sa incerci un trial cu valorile astea"
study.enqueue_trial(
    {
        "subsample": 0.9,
        "colsample_bytree": 0.6,
        "min_child_weight": 2,
    }
)

study.enqueue_trial(
    {
        "subsample": 0.75,
        "colsample_bytree": 0.8,
        "min_child_weight": 5,
    }
)

import logging
import sys

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study.optimize(objective, n_trials=100, timeout=600)  # Incepem procesul: de 100 de ori, rulam functia "objective" si schimbam hiperparametrii alesi de fiecare data (timeout: 600 secunde)

[I 2025-11-07 18:18:39,620] Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2}. Best is trial 0 with value: 0.8225255972696246.


Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2}. Best is trial 0 with value: 0.8225255972696246.
Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2}. Best is trial 0 with value: 0.8225255972696246.
Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2}. Best is trial 0 with value: 0.8225255972696246.
Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2}. Best is trial 0 with value: 0.8225255972696246.
Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2}. Best is trial 0 with value: 0.8225255972696246.
Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree

[I 2025-11-07 18:18:39,973] Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_bytree': 0.8, 'min_child_weight': 5}. Best is trial 1 with value: 0.8361774744027304.


Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_bytree': 0.8, 'min_child_weight': 5}. Best is trial 1 with value: 0.8361774744027304.
Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_bytree': 0.8, 'min_child_weight': 5}. Best is trial 1 with value: 0.8361774744027304.
Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_bytree': 0.8, 'min_child_weight': 5}. Best is trial 1 with value: 0.8361774744027304.
Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_bytree': 0.8, 'min_child_weight': 5}. Best is trial 1 with value: 0.8361774744027304.
Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_bytree': 0.8, 'min_child_weight': 5}. Best is trial 1 with value: 0.8361774744027304.
Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_

[I 2025-11-07 18:18:40,292] Trial 2 finished with value: 0.8395904436860068 and parameters: {'subsample': 0.8993884067878952, 'colsample_bytree': 0.864539932624941, 'min_child_weight': 6.59442452123882}. Best is trial 2 with value: 0.8395904436860068.


...

Trial 97 finished with value: 0.8327645051194539 and parameters: {'subsample': 0.7949160974275296, 'colsample_bytree': 0.8082528041781827, 'min_child_weight': 4.09027596624195}. Best is trial 67 with value: 0.8464163822525598.
Trial 97 finished with value: 0.8327645051194539 and parameters: {'subsample': 0.7949160974275296, 'colsample_bytree': 0.8082528041781827, 'min_child_weight': 4.09027596624195}. Best is trial 67 with value: 0.8464163822525598.
Trial 97 finished with value: 0.8327645051194539 and parameters: {'subsample': 0.7949160974275296, 'colsample_bytree': 0.8082528041781827, 'min_child_weight': 4.09027596624195}. Best is trial 67 with value: 0.8464163822525598.
Trial 97 finished with value: 0.8327645051194539 and parameters: {'subsample': 0.7949160974275296, 'colsample_bytree': 0.8082528041781827, 'min_child_weight': 4.09027596624195}. Best is trial 67 with value: 0.8464163822525598.
Trial 97 finished with value: 0.8327645051194539 and parameters: {'subsample': 0.79491609742

[I 2025-11-07 18:19:15,661] Trial 98 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.6665238547532215, 'colsample_bytree': 0.8799526928332981, 'min_child_weight': 5.38740680224388}. Best is trial 67 with value: 0.8464163822525598.


Trial 98 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.6665238547532215, 'colsample_bytree': 0.8799526928332981, 'min_child_weight': 5.38740680224388}. Best is trial 67 with value: 0.8464163822525598.
Trial 98 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.6665238547532215, 'colsample_bytree': 0.8799526928332981, 'min_child_weight': 5.38740680224388}. Best is trial 67 with value: 0.8464163822525598.
Trial 98 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.6665238547532215, 'colsample_bytree': 0.8799526928332981, 'min_child_weight': 5.38740680224388}. Best is trial 67 with value: 0.8464163822525598.
Trial 98 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.6665238547532215, 'colsample_bytree': 0.8799526928332981, 'min_child_weight': 5.38740680224388}. Best is trial 67 with value: 0.8464163822525598.
Trial 98 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.66652385475

[I 2025-11-07 18:19:16,130] Trial 99 finished with value: 0.8395904436860068 and parameters: {'subsample': 0.7685368107354739, 'colsample_bytree': 0.9888057744245403, 'min_child_weight': 4.66636644290043}. Best is trial 67 with value: 0.8464163822525598.


Trial 99 finished with value: 0.8395904436860068 and parameters: {'subsample': 0.7685368107354739, 'colsample_bytree': 0.9888057744245403, 'min_child_weight': 4.66636644290043}. Best is trial 67 with value: 0.8464163822525598.
Trial 99 finished with value: 0.8395904436860068 and parameters: {'subsample': 0.7685368107354739, 'colsample_bytree': 0.9888057744245403, 'min_child_weight': 4.66636644290043}. Best is trial 67 with value: 0.8464163822525598.
Trial 99 finished with value: 0.8395904436860068 and parameters: {'subsample': 0.7685368107354739, 'colsample_bytree': 0.9888057744245403, 'min_child_weight': 4.66636644290043}. Best is trial 67 with value: 0.8464163822525598.
Trial 99 finished with value: 0.8395904436860068 and parameters: {'subsample': 0.7685368107354739, 'colsample_bytree': 0.9888057744245403, 'min_child_weight': 4.66636644290043}. Best is trial 67 with value: 0.8464163822525598.
Trial 99 finished with value: 0.8395904436860068 and parameters: {'subsample': 0.76853681073

In [73]:
print(study.best_trial.params)  # cel mai bun trial (hiperparametrii cu cel mai mare accuracy)
best_params = study.best_trial.params  

best_params.update({                  # add what's still needed for training the model
    "objective": "multi:softprob",
    "num_class": 4,
    "eval_metric": "mlogloss"
})
dtrain = xgb.DMatrix(X_train, label=Y_train)
dtest = xgb.DMatrix(X_test)


tuned_model = xgb.train(best_params, dtrain, num_boost_round=100)
y_pred_tuned = tuned_model.predict(dtest)

# y_pred_tuned va arata cv de genul [ [0.2, 0.5, 0.1, 0.2],
#                                     [0.6, 0.1, 0.1, 0.2],
#                                     [0.1, 0.3, 0.2, 0.4],
#                                     ...                  ]   
# 
# ce inseamna: pt fiecare rand din dtest s-a asignat cate o probabilitate pt ce pret va fi;
# gen noi avem ["cheap", "affordable", "expensive", "billionaire type of shit"]
#   ==> exemplul nostru: 0.2: cheap, 0.5: affordable, 0.1: expensive, 0.2: billionaire type of shit
#
# ce face urmatoarea linie de cod: pastram doar pretul cu cea mai mare probabilitate (asta ne trebe)
# pe exemplul de inainte:           [1,0,3,...]  
y_pred_tuned = np.argmax(y_pred_tuned, axis=1)

print()   #newline
print(f" Accuracy of prediction (untuned model): {accuracy_score(Y_test, y_pred)}")
print(f" Accuracy of prediction (tuned model): {accuracy_score(Y_test, y_pred_tuned)}")

{'subsample': 0.776041830016811, 'colsample_bytree': 0.8528163488440733, 'min_child_weight': 4.340467509933016}

 Accuracy of prediction (untuned model): 0.8088737201365188
 Accuracy of prediction (tuned model): 0.8361774744027304
